Useful link: https://tutorials.geemap.org/AssetManagement/extract_values/
https://tutorials.geemap.org/AssetManagement/extract_values/
https://www.youtube.com/watch?v=bzjlTbEByZo

## Test code to extract data from map interactively

In [1]:
import os
import ee
import pickle
import geemap
import geefun as gf
import geopandas as gpd
import geemap.colormaps as cm

from tqdm import tqdm

/Users/szalam/cfprj/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Extract land use at water quality measurement buffers

In [2]:
m3 = geemap.Map()

collection = ee.ImageCollection("USDA/NASS/CDL") \
    .filterDate('2008-01-01', '2021-12-31') \
    .select('cropland')

# Convert the image collection to an image.
image = collection.toBands()

palette = cm.palettes.ndvi

cdl_vis = {
  'min': 0.0,
  'max': 255.0,
  'palette': palette
}

m3.addLayer(image, {}, 'CDL Time-series')
m3.addLayer(image.select(0), cdl_vis, 'CDL VIS')

m3

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
work_dir = os.path.expanduser('/Volumes/GoogleDrive/Shared drives/GWAttribution/data/processed')
in_shp = os.path.join(work_dir, 'Well_buffer_shape/Well_buffer_2mil_0_10.shp')

in_fc = geemap.shp_to_ee(in_shp)

m3.addLayer(in_fc, {}, 'WQ buffer')


In [4]:
import pandas as pd
import numpy as np

def cdl_2_faunt():

    '''
    Classify crop types from CDL to the faunt (2009), schmid (2004) scheme 
    CDL classes: https://developers.google.com/earth-engine/datasets/catalog/USDA_NASS_CDL
    Faunt kc and classes: https://water.usgs.gov/GIS/metadata/usgswrd/XML/pp1766_fmp_parameters.xml 
    Dict Key is the Faunt class (int)     
    Dict Value is the CDL category (string)
    The faunt class = CDL category is shown at the top of each k:v pair. 
    '''

    data = {
        # Water = water(83), wetlands(87), Aquaculture(92), Open Water(111), Perreniel Ice / Snow (112)
        1 : ["83", "87", "92", "111", "112"], 
        # Urban = developed high intensity(124), developed medium intensity(123)
        2 : ["124", "123"], 
        # Native = grassland/pasture(176), Forest(63), Shrubs(64), barren(65, 131), Clover/Wildflowers(58)
        # Forests (141 - 143), Shrubland (152), Woody Wetlands (190), Herbaceous wetlands (195)
        3 : ["176","63","64", "65", "131","58", "141", "142", "143", "152", "190", "195"], 
        # Orchards, groves, vineyards = 
        4 : [""],
        # Pasture / hay = other hay / non alfalfa (37)
        5 : ["37"],
        # Row Crops = corn (1), soybeans (5),Sunflower(6) sweet corn (12), pop corn (13), double winter/corn (225), 
        # double oats/corn(226), double barley/corn(237), double corn / soybeans
        6 : ["1", "5", "6", "12", "13", "225", "226", "237", "239"] ,
        # Small Grains = Spring wheat (23), winter wheat (24), other small grains (25), winter wheat / soybeans (26), 
        # rye (27), oats (28), Millet(29), dbl soybeans/oats(240)
        7 : ["23", "24", "25", "26", "27", "28", "29", "240"] ,
        # Idle/fallow = Sod/Grass Seed (59), Fallow/Idle Cropland(61), 
        8 : ["59","61"],
        # Truck, nursery, and berry crops = 
        # Blueberries (242), Cabbage(243), Cauliflower(244), celery (245), radishes (246), Turnips(247)
        # Eggplants (249), Cranberries (250), Caneberries (55), Brocolli (214), Peppers(216), 
        # Greens(219), Strawberries (221), Lettuce (227), Double Lettuce/Grain (230 - 233)
        9 : ["242", "243", "244", "245", "246", "247", "248", "249", "250", "55", "214", "216","219","221", "227", "230", "231", "232", "233"], 

        # Citrus and subtropical = Citrus(72), Oranges (212), Pommegranates(217)
        10 : ["72", "212", "217"] ,

        # Field Crops = 
        # Peanuts(10),Mint (14),Canola (31),  Vetch(224),  Safflower(33) , RapeSeed(34), 
        # Mustard(35) Alfalfa (36),Camelina (38), Buckwheat (39), Sugarbeet (41), Dry beans (42), Potaoes (43)
        # Sweet potatoes(46), Misc Vegs & Fruits (47), Cucumbers(50)
        # Chick Peas(51),Lentils(52),Peas(53),Tomatoes(54)Hops(56),Herbs(57),Carrots(206),
        # Asparagus(207),Garlic(208), Cantaloupes(209), Honeydew Melons (213), Squash(222), Pumpkins(229), 

        11 : ["10",  "14", "224", "31","33", "34", "35", "36", "38", "39", "41", "42", "43", "46", "47", "48" ,
              "49", "50", "51", "52", "53", "54",  "56", "57","206","207", "208", "209","213","222", "229"] ,

        # Vineyards = Grapes(69)
        12 : ["69"],
        # Pasture = Switchgrass(60)
        13 : ["60"],
        # Grain and hay = Sorghum(4), barley (21), Durham wheat (22), Triticale (205), 
        # Dbl grain / sorghum (234 - 236), Dbl 
        14 : ["4", "21", "22", "205", "234", "235", "236"],
        # livestock feedlots, diaries, poultry farms = 
        15 : [""],

        # Deciduous fruits and nuts = Pecans(74), Almonds(75), 
        # Walnuts(76), Cherries (66), Pears(77), Apricots (223), Apples (68), Christmas Trees(70)
        # Prunes (210), Plums (220), Peaches(67), Other Tree Crops (71), Pistachios(204), 
        # Olives(211), Nectarines(218), Avocado (215)
        16 : ["74", "75", "76","66","77", "223", "68", "210", "220", "67", "70", "71", "204", "211","215","218"],

        # Rice = Rice(3)
        17 : ["3"],
        # Cotton = Cotton (2) , Dbl grain / cotton (238-239)
        18 : ["2", "238", "239"], 
        # Developed = Developed low intensity (122) developed open space(121)
        19 : ["122", "121"],
        # Cropland and Pasture
        20 : [""],
        # Cropland = Other crops (44)
        21 : ["44"], 
        # Irrigated row and field crops = Woody Wetlands (190), Herbaceous wetlands(195)
        22 : [""] # ["190", "195"] 
        }

    return data

def map_cdl2fmp(dictionary,array):
    '''maps values on cdl image to the fmp'''
    
    mapping = dictionary.copy()
    
    vec1 = []
    vec2 = []

    for k,v in mapping.items():
        for i in v:
            if i == "":
                continue
            else:
                vec1.append(int(i))
                vec2.append(int(k))
                
    out_im = np.zeros_like(array)
    for k,v in dict(zip(vec1,vec2)).items():
        out_im[array==k] =v
    
    return out_im

def dict2arr(data_dict, var_name):
    '''converts ee dictionary output from .getInfo() to a numpy array. Wraps array_from_df'''
    
    data = data_dict[var_name]
    lats = data_dict['latitude']
    lons = data_dict['longitude']

    df = pd.DataFrame([data,lats,lons]).T
    df.columns = [var_name, "latitude", 'longitude']
    arr = array_from_df(df, var_name)
    
    return arr

def array_from_df(df, variable):    

    '''
    Convets a pandas df with lat, lon, variable to a numpy array 
    '''

    # get data from df as arrays
    lons = np.array(df.longitude)
    lats = np.array(df.latitude)
    data = np.array(df[variable]) # Set var here 

    # get the unique coordinates
    uniqueLats = np.unique(lats)
    uniqueLons = np.unique(lons)

    # get number of columns and rows from coordinates
    ncols = len(uniqueLons)    
    nrows = len(uniqueLats)

    # determine pixelsizes
    ys = uniqueLats[1] - uniqueLats[0] 
    xs = uniqueLons[1] - uniqueLons[0]

    # create an array with dimensions of image
    arr = np.zeros([nrows, ncols], np.float32)

    # fill the array with values
    counter =0
    for y in range(0,len(arr),1):
        for x in range(0,len(arr[0]),1):
            if lats[counter] == uniqueLats[y] and lons[counter] == uniqueLons[x] and counter < len(lats)-1:
                counter+=1
                arr[len(uniqueLats)-1-y,x] = data[counter] # we start from lower left corner

    return arr

In [ ]:
gdf = gpd.read_file(work_dir +  '/Well_buffer_shape/Well_buffer_2mil.shp')
gdf = gdf.to_crs('epsg:4326')
area_shp = gdf['geometry']

crop_df = pd.DataFrame(range(1,254))
crop_df.columns = ['Crop_id']

# reading cdl crop names and corresponding ids
crp_leg = pd.read_csv('/Volumes/GoogleDrive/Shared drives/GWAttribution/data/raw/cdl_legend.csv')


# join name with crop_df dataframe
crop_df = crop_df.merge(crp_leg,how='outer',left_on=['Crop_id'],right_on=['Crop_id'])


cdl_all = []
lap=100
start_from = n =1200
for reg_range in tqdm(range(start_from,area_shp.shape[0])):
    
    crop_df2 = crop_df.copy()
    crop_df2['WELL ID'] = gdf['WELL ID'][reg_range]
    reg = gf.gp_to_ee_poly(area_shp[reg_range:reg_range+1])

    for yr in range(2008,2020):
        #print('Processing year: ' + str(yr))
        #extracting land use types as numbers from gee
        cdl_dict = gf.cdl_pr(yr,reg)

        #list the unique land use types
        crop_all = gf.unique(cdl_dict['cropland'])

        # counting grid numbers with specific land uses and calculating area in m^2. 30 m cell size
        df_crop = gf.crop_all_area(crop_all,cdl_dict,s_grid=30,attrb = 'cropland')
        df_crop2 = df_crop.T
        df_crop2.columns = ['Crop_id', str('Area_'+ str(yr))]
        df_crop2 = df_crop2.sort_values('Crop_id')

        # join to common dataframe
        crop_df2 = crop_df2.merge(df_crop2,how='outer',left_on=['Crop_id'],right_on=['Crop_id'])
    
#     cdl_all = pd.concat([cdl_all, crop_df])
    cdl_all.append(crop_df2)
    
    if len(cdl_all)==lap:
        with open(f"/Volumes/GoogleDrive/Shared drives/GWAttribution/data/processed/CDL/cdl_at_buffers/cdl_buf_2mi_{n-lap+1}_{n}", "wb") as fp:   #Pickling
            pickle.dump(cdl_all, fp)
        cdl_all = []
    
    n = n+1
        


 11%|████▎                                 | 206/1831 [13:06<1:32:50,  3.43s/it]

In [13]:
type(cdl_all)

list

In [138]:
cdl_all[0]

,Crop_id,Crop_type,Legend_color,WELL ID,Area_2008,Area_2009,Area_2010,Area_2011,Area_2012,Area_2013,Area_2014,Area_2015,Area_2016,Area_2017,Area_2018,Area_2019
0,1,Corn,ffd300,NO3_1023107,3.0330,3.0816,5.1291,4.9581,3.6126,3.8340,2.4876,2.6307,2.6874,1.2114,1.4598,0.9360
1,2,Cotton,ff2626,NO3_1023107,0.3969,0.0288,1.8279,1.8369,1.0458,0.3609,1.3734,0.9000,0.9603,1.0548,0.3699,0.0873
2,3,Rice,00a8e2,NO3_1023107,NaN,NaN,NaN,NaN,NaN,NaN,0.0009,NaN,NaN,NaN,NaN,0.0027
3,4,Sorghum,ff9e0a,NO3_1023107,NaN,NaN,NaN,NaN,0.0144,0.0027,0.0045,0.0612,0.0036,NaN,0.0378,0.0090
4,5,Soybeans,267000,NO3_1023107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,250,Cranberries,ff6666,NO3_1023107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,251,NaN,NaN,NO3_1023107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251,252,NaN,NaN,NO3_1023107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252,253,NaN,NaN,NO3_1023107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
area_shp.shape[0]

3031

In [102]:
crop_df

,Crop_id,Crop_type,Legend_color,Area_2008,Area_2009,Area_2010,Area_2011,Area_2012,Area_2013,Area_2014,Area_2015,Area_2016,Area_2017,Area_2018,Area_2019
0,1,Corn,ffd300,0.0342,0.0198,0.5625,0.1332,0.4725,0.0954,0.0207,0.2241,0.0198,0.0009,0.1458,0.0972
1,2,Cotton,ff2626,0.0171,0.0135,0.2124,0.1503,0.1125,0.0063,0.0648,0.0117,0.1665,0.0009,0.0945,0.1350
2,3,Rice,00a8e2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Sorghum,ff9e0a,NaN,NaN,0.0009,0.0306,0.0009,NaN,0.0270,0.0180,NaN,NaN,0.0027,0.0036
4,5,Soybeans,267000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,250,Cranberries,ff6666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251,252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252,253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
